In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# We will use the scipy library to evolve
# our ODEs (below).
from scipy.integrate import solve_ivp

## Background ##

In 1801 a young Gauss became an overnight sensation when he rediscovered the minor planet [Ceres](https://en.wikipedia.org/wiki/Ceres_(dwarf_planet)), which had been "lost" by its discoverer (Piazzi) earlier that year.  You can read more of the history and his methodology [here](https://math.berkeley.edu/~mgu/MA221/Ceres_Presentation.pdf) and [here](http://sites.math.rutgers.edu/~cherlin/History/Papers1999/weiss.html).  In brief he was clever, he calculated by hand (for over 100 hours), and he invented the method of least squares.  We're not as clever and we'll use a computer, but we'll follow his lead and use the method of least squares too.  Hopefully this will take less than 100 hours!

### Model ###

We integrate the (very simple) ODEs for motion of a test particle under a central force.  We use the ```scipy``` routine ```solve_ivp``` to evolve the particle starting from initial positions ($x$) and velocities ($v$) then "observe" it from the Earth -- which we simplify as a point moving in the $x-y$ plane at radius 1 with 1 revolution per unit time (i.e. our length units are AU and our time units years).  One could certainly imagine a more complex set-up where we included a proper Earth motion and even perturbations from the other (major) planets.  The ODE isn't very hard to set up.

If the Earth were on a circular orbit, then $v^2=GM_\odot/r$.  For a period of 1 unit and a radius of 1 unit this would imply $v=2\pi$ and hence $GM_\odot=(2\pi)^2$.

Fill in the routine below.

In [ ]:
# Set up code which will integrate the orbit given a initial conditions and
# "observe" our object from the Earth.
# We'll idealize this problem dramatically, putting the Earth on a circular
# orbit in the x-y plane and ignoring the influence of everything but the Sun.
def derivs(t,y):
    """The derivatives for the ODE integration (see below)."""
    kSol   = 4*np.pi**2                # GM in Msun, AU, yr units. and astronomical units.
    r2     = np.sum(y[:3]**2)          # Squared radius.
    nhat   = y[:3]/(np.sqrt(r2)+1e-20) # Avoid divide by zero.
    dy     = np.empty_like(y)
    # To avoid round-off error and avoid Ceres-Sun scattering in case our
    # orbit gets too close to the origin, we add a small positive constant
    # to our squared radius.
    # The derivatives are just dr/dt=v and dv/dt=-GM/r^2 nhat:
    ...
    ...
    return(dy)
    #
def predict_angles(r,v,tobs):
    """Given initial positions and velocities (r,v), predict the angular positions
       as viewed from the Earth at observation times tobs."""
    # We pack the position and velocity into a 6-vector, y, and integrate the EOM.
    y0  = np.append(r,v)
    res = solve_ivp(derivs,[tobs[0],tobs[-1]],y0,t_eval=tobs).y.T
    # We've implicitly assumed this integration would succeed, so we haven't
    # checked the status and we're throwing away everything except the solution.
    # The transpose is just to get a (Ntime,6) shaped-array rather than (6,Ntime).
    # Now work out the Earth's position and subtract it from each observation
    # to get a relative vector.
    omegat = 2*np.pi*tobs
    earth  = np.vstack( (np.cos(omegat),np.sin(omegat),0*omegat) ).T
    rel    = res[:,:3] - earth
    # and from the relative vector work out theta and phi.
    rr     = np.sqrt( np.sum( rel[:,:3]**2, axis=1 ) )
    thta   = np.arccos( rel[:,2]/(rr+1e-30))
    phi    = np.arctan2(rel[:,1],rel[:,0])
    # We return the "true" path of this orbit as well, in case we want
    # to look at it in 3D or something later.
    return( (thta,phi,res[:,:3]) )

In [ ]:
# Generate a data vector by simulating an orbit, observing it and then adding noise.
r0   = np.array([1.0,5.0,5.0])
vcir = (2*np.pi)/(np.sum(r0**2))**0.25  # Velocity for a circular orbit: Sqrt[GM/r].
v0   = vcir * np.array([0.5,0.5,0.5])   # Hold this fixed for now...
tobs = np.linspace(0.,10.,100)          # Must contain IVP, i.e. t=0.
print("Initial conditions: ",r0," ",v0)
# Generate a true path and add noise.
thta,phi,pth = predict_angles(r0,v0,tobs)
thta += 0.01 * np.random.normal(size=len(thta))
phi  += 0.01 * np.random.normal(size=len(phi ))

In [ ]:
deg    = 180./np.pi
fig,ax = plt.subplots(1,1,figsize=(5,4))
cax    = ax.scatter(phi*deg,(np.pi/2-thta)*deg,c=tobs)
plt.colorbar(cax)
ax.set_xlabel('Azimuth')
ax.set_ylabel('Altitude')

This seems quite complex, partly due to the fact that we are viewing it from the earth which is rotating around the Sun once a year.  So there are ten rotations of the "viewing platform" built into this orbit.

## Orbit determination through minimization ##

To find the orbit of Ceres now becomes a minimization problem, in 6D (initial position and velocity components).  Let us define $\chi^2$ of a 6D vector $(r,v)$.  For simplicity we'll assume $\theta$ and $\phi$ have constant errors of 0.01 radians, since this is what we put in!  Note this isn't super physical, but serves our purposes.

Finish the $\chi^2$ routine -- this should return $\chi^2$ given the 6 parameters packed as ```r=pars[:3]``` and ```v=pars[3:]```, i.e. $(x,y,z,v_x,v_y,v_z)$.

In [ ]:
def chi2(pars):
    """Returns the chi^2 for fitting the data given the 6 "parameters" (r,v)."""
    ...

## Minimization without gradient information ##

When deciding how to minimize this function numerically we first need to decide whether or not we have gradient information.  If we do, then the minimization should be much more efficient because we have a guess as to which direction to step to find the minimum (i.e. opposite the gradient direction).  This becomes a bigger and bigger deal the higher the dimension of the space.  Speaking roughly, it take $\mathcal{O}(N^2_{\rm par})$ steps to find the minimum without gradient information and  $\mathcal{O}(N_{\rm par})$ with it.

Without gradient information some common choices are:

* Random flailing around.
* Pattern search.
* Simplex.
* Powell.
* Levenburg-Marquardt.
* Accelerated particle swarm optimization.

Let's use Powell's method, as implemented in ```scipy```.  Compare the minimum that Powell's method finds with the true initial conditions.  How many function evaluations did we need?

In [ ]:
# Use the scipy minimization routine(s).
from scipy.optimize import minimize

In [ ]:
init_guess = [1.,5.,5.,1.17559391, 1.17559391, 1.17559391]
res = minimize(chi2,init_guess,method='powell')
print(res)

In our case evaluations of $\chi^2$ are relatively "cheap", but what if we had to do some complex N-body integration each time or we needed to do this an enormous number of times?

With our initial conditions in hand we can plot the best-fit curve through the points.  The best-fit line will help to clarify what is actually happening in terms of the orbit.

In [ ]:
pred_thta,pred_phi,pred_path = predict_angles(res.x[:3],res.x[3:],tobs)
# Plot the data.
fig,ax = plt.subplots(1,1,figsize=(6,4))
cax    = ax.scatter(phi*deg,(np.pi/2-thta)*deg,c=tobs)
ax.plot(pred_phi*deg,(np.pi/2-pred_thta)*deg,ls='-',color='C0')
ax.set_xlabel('Azimuth')
ax.set_ylabel('Altitude')
plt.colorbar(cax)

Let's extend the curve to later times to "predict" the orbit, and use a lot of points to make the curve smooth.  What's happening soon after the last point plotted above?

In [ ]:
textra = np.linspace(0,20,2000)
pred_thta,pred_phi,pred_path = predict_angles(res.x[:3],res.x[3:],textra)
# Plot the data.
fig,ax = plt.subplots(1,1,figsize=(6,4))
cax    = ax.scatter(phi*deg,(np.pi/2-thta)*deg,c=tobs)
ax.plot(pred_phi*deg,(np.pi/2-pred_thta)*deg,ls='-',color='C0')
ax.set_xlabel('Azimuth')
ax.set_ylabel('Altitude')
plt.colorbar(cax)

## Minimization with gradient information ##

With gradient information things become easier.  The way we would probably proceed with our problem is to use an 
[automatic differentation](https://en.wikipedia.org/wiki/Automatic_differentiation) library (or language) to generate the gradients of our function for us.  Common tools include AutoDiff, Stan or JAX or languages such as Julia.  The basic idea behind such tools is that since the computer is performing some sequence of basic operations to compute $\chi^2$, and since we know (a) how to compute the derivative of operations like $x^2$ or $\arctan x$ and (b) we know how to use the chain rule to "compose" multiple operations it's an algorithmic matter to compute the derivatives automagically.

To avoid introducing more libraries and tools, let's simplify our problem so that we can do the derivatives by hand.

### Simplified problem ###

We want to simplify this problem as much as possible.  As a first step let's destroy the earth (harsh, but in a good cause!) and move to the Sun (i.e. the origin).  Secondly let's decide that we will only consider observations close to the $x-y$ plane and near $y=0$, so that the sky looks flat to us and we can avoid the arctangents and things and just look in the plane of the sky.  We can always rotate our coordinates so any given point lies along $(1,0,0)$.

Further, suppose Ceres is distant from the Sun, so its orbital time is long, and suppose that the observations are taken over a short time period.  Now we can Taylor series expand the path of Ceres around the central observation:
$$
  \vec{x}(t) \simeq \vec{x}_0 + (t-t_0)\left.\frac{d\vec{x}}{dt}\right|_0
    + \frac{1}{2}(t-t_0)^2 \left.\frac{d^2\vec{x}}{dt^2}\right|_0
    = \vec{x}_0 + (t-t_0)\vec{v}_0 - \frac{k\hat{x}_0}{2r_0^2}(t-t_0)^2
$$
where $r_0^2=|\vec{x}_0|^2$.  For fixed observations $\vec{x}(t_i)$ is a function of $\vec{x}_0$ and $\vec{v}_0$.

We will imagine we are given $\ge 3$ observations of the projected position on the sky, at $t<t_0$, $t=t_0$ and $t>t_0$.  By choice of time origin (epoch) we can always make $t_0=0$.  Note we need at least 3 positions, because we need to solve for $\vec{x}_0$ and $\vec{v}_0$ which is 6 numbers and we get 2 constraints per observation.  We orient our coordinates such that $\vec{x}_0\propto\widehat{x}$, and then our data are the $y$ and $z$ positions at the earlier and later times, converted to angles (in the small-angle approximation) by dividing by $|\vec{x}|$.  This gives 4 data points and we need to solve for the length of $\vec{x}_0$ and for $\vec{v}_0$, i.e. 4 unknowns.  We have $r_0$ as the length of $\vec{x}_0$ so our free parameters are $(r_0,v_{x0},v_{y0},v_{z0})$.

Write some code to return $\vec{x}$ given $t$ and the parameters $\vec{\alpha}=(r_0,v_{x0},v_{y0},v_{z0})$.

In [ ]:
def prediction(tt,alpha):
    """Returns the predicted \vec{x}.  Note r(t)=|x(t)|."""
    kSol= 4*np.pi**2                # GM in Msun, AU, yr units. and astronomical units.
    x0  = np.array([alpha[0],0,0])
    v0  = alpha[1:]
    ...
    ...

Let us generate some data, given some "true" parameters, $\alpha$, and some observation times.  We'll call $\theta=y/r$ and $\phi=z/r$ just to have some names for them, though note these aren't the usual polar angles.  We only actually need two "additional" observations beyond the one at $t_0$ but adding a few extra helps constrain the problem, so why don't we choose 3 "extra" observations (4 total).

Also, we won't add any "noise" to these observations so we expect that we will recover the "true" initial conditions at $\chi^2=0$ and this should be independent of what we choose for our error bar in $\chi^2$.

In [ ]:
alpha_true= np.array([2.0,0.0,1.0,0.5])
obstime   = np.array([-0.50,0.1,0.75])
tobs,pobs = np.zeros_like(obstime),np.zeros_like(obstime)
for i,tt in enumerate(obstime):
    pred    = prediction(tt,alpha_true)
    rr      = np.sqrt(np.sum(pred**2))
    tobs[i] = pred[1]/rr
    pobs[i] = pred[2]/rr
#
print("time=",obstime)
print("tobs=",tobs)
print("pobs=",pobs)
# Set the "observation accuracy":
sigma = 0.001

Our $\chi^2$ is
$$
  \chi^2 = \frac{1}{\sigma^2} \sum_{i\ne 0}
  \left(\frac{y(t_i)}{r(t_i)}-\theta_i^{\rm obs}\right)^2 +
  \left(\frac{z(t_i)}{r(t_i)}-\phi_i^{\rm obs}\right)^2
$$
with
$$
  \vec{x}(t) = r_0\widehat{x} + t\vec{v}_0 - \frac{k\widehat{x}}{2r_0^2}t^2
  = \left( \begin{array}{c} \rho + tv_{x0} \\ tv_{y0} \\ tv_{z0} \end{array}\right)
  \quad , \quad
  \left|\vec{x}(t)\right| = t\sqrt{ (\rho/t+v_{x0})^2+v_{y0}^2+v_{z0}^2 }
$$
where I've written $\rho=r_0-(k/2r_0^2)t^2$ for simplicity.

Write code to return $\chi^2$ for a 4-dimensional vector $\alpha$ with the parameters ordered as $r_0$, $v_{x0}$, $v_{y0}$ and $v_{z0}$.

In [ ]:
def chi2(alpha):
    """Computes chi2 at point alpha."""
    c2 = 0.0
    # There are contributions to chi2 from each time:
    for i,tt in enumerate(obstime):
        pred  = prediction(tt,alpha)
        rr    = np.sqrt(np.sum(pred**2))
        ...
        ...
    return(c2/sigma**2)

The derivative of our $\chi^2$ value can be built up in pieces using the chain rule and the simple building blocks that we have.  If I call the parameter $\alpha$, (i.e. $\alpha$ is one of $r_0$, $v_{x0}$, etc) then
$$
  \frac{\partial\chi^2}{\partial\alpha} =
  \frac{2}{\sigma^2}\sum_{i\ne 0} 
  \left(\frac{y(t_i)}{r(t_i)}-\theta_i^{\rm obs}\right)\frac{\partial y/r}{\partial\alpha} +
  \left(\frac{z(t_i)}{r(t_i)}-\phi_i^{\rm obs}\right)\frac{\partial z/r}{\partial\alpha}
$$
and for example
$$
  \frac{\partial y/r}{\partial\alpha} = \frac{1}{r}\frac{\partial y}{\partial\alpha}
  -\frac{y}{r^2}\frac{\partial r}{\partial\alpha}
$$
with the other terms being similar.  Now $\partial y/\partial\alpha$ is very simple, since $y=tv_{y0}$.  This means $\partial y/\partial\alpha=0$ unless $\alpha=v_{y0}$ in which case it is $t$.  The slightly more complicated case is $\partial r/\partial\alpha$, but this is also reasonably straightforward:
$$
  \frac{\partial r}{\partial\alpha} = \frac{t^2}{2|\vec{x}(t)|}
  \frac{\partial\left[(\rho/t+v_{x0})^2+v_{y0}^2+v_{z0}^2\right]}{\partial\alpha}
$$
and the final derivative is very easy for all but $\alpha=r_0$.

Each step is straightforward, if slightly tedious, and we can build this up in code by computing each of the building blocks.  The code is easier to read, to understand and to debug if you try to break it up in pieces like we have above rather than attempting to grind out all of the formulae and type in one huge code block.

Finish off the routine below that computes $\partial\chi^2/\partial\alpha_i$ for parameters $\alpha_i$ ordered as $\vec{\alpha}=(r_0,v_{x0},v_{y0},v_{z0})$.

In [ ]:
def derivs(alpha):
    """Computes the derivatives of chi2 with respect to the 4 parameters, at the point alpha."""
    # Make some storage for the derivatives and define sigma.
    dd  = np.zeros(4) # The derivatives go here.
    kSol= 4*np.pi**2  # GM in Msun, AU, yr units. and astronomical units.
    # There are contributions to dchi2 from each time:
    for i,tt in enumerate(obstime):
        pred  = prediction(tt,alpha)
        rr    = np.sqrt(np.sum(pred**2))
        rho   = alpha[0] - kSol/2/alpha[0]**2*tt**2
        yor   = pred[1]/rr
        zor   = pred[2]/rr
        # Start with v_y0 and v_z0, i.e. the last two parameters.
        # First do the v_y0 derivative.
        dy    = tt
        dz    = 0
        dr    = tt**2/rr * alpha[2]
        dyor  = 1/rr*dy - pred[1]/rr**2*dr
        dzor  = 1/rr*dz - pred[2]/rr**2*dr
        dd[2]+= (yor-tobs[i])*dyor + (zor-pobs[i])*dzor
        # and then the v_z0 derivative.
        ...
        # Now the v_x0 derivative, which is only slightly more complicated.
        ...
        # and finally the r_0 derivative, which is the most complicated.
        ...
    # Finally multiply by the overall prefactor and return the answer.
    dd *= 2/sigma**2
    return(dd)

### Comparing minimizations ###

Now let us compare minimizations with and without gradients.  We'll start with the no-gradient version like we did above.

Run the minimizer and compare to the "true" value.  How many function evaluations (```nfev```) does it take?

In [ ]:
init_guess = np.array([1.9,0.0,1.1,0.4])
res = minimize(chi2,init_guess,method='powell')
#
for i in range(init_guess.size):
    print("alpha[{:d}]={:8.4f} vs. truth {:8.4f}".\
          format(i,res.x[i],alpha_true[i]))
print("Taking {:d} function evaluations.".format(res.nfev))
print("Final chi2={:e}\n".format(chi2(res.x)))
# Compare to the data
for i,tt in enumerate(obstime):
    pred    = prediction(tt,res.x)
    rr      = np.sqrt(np.sum(pred**2))
    print("tobs[{:d}]={:10.5f} vs predicted {:10.5f}".format(i,tobs[i],pred[1]/rr))
    print("pobs[{:d}]={:10.5f} vs predicted {:10.5f}".format(i,pobs[i],pred[2]/rr))
# and we can actually look at the underlying 3D positions as well.
print("\n")
for i,tt in enumerate(obstime):
    truth   = prediction(tt,alpha_true)
    pred    = prediction(tt,res.x)
    #
    truth   = ["{:10.5f}".format(x) for x in truth]
    pred    = ["{:10.5f}".format(x) for x in pred]
    #
    outstr  = "x({:6.2f})=".format(tt)
    for itm in truth: outstr += itm
    outstr += " vs predicted "
    for itm in pred: outstr += itm
    print(outstr)

Now try this with a minimizer than can use gradient information.  We'll start from exactly the same place with the same data.

In [ ]:
res = minimize(chi2,init_guess,jac=derivs,method='cg')
#
for i in range(init_guess.size):
    print("alpha[{:d}]={:8.4f} vs. truth {:8.4f}".\
          format(i,res.x[i],alpha_true[i]))
print("Taking {:d} function evaluations.".format(res.nfev))
print("Final chi2={:e}\n".format(chi2(res.x)))
#
for i,tt in enumerate(obstime):
    pred    = prediction(tt,res.x)
    rr      = np.sqrt(np.sum(pred**2))
    print("tobs[{:d}]={:10.5f} vs predicted {:10.5f}".format(i,tobs[i],pred[1]/rr))
    print("pobs[{:d}]={:10.5f} vs predicted {:10.5f}".format(i,pobs[i],pred[2]/rr))
# and we can actually look at the underlying 3D positions as well.
print("\n")
for i,tt in enumerate(obstime):
    truth   = prediction(tt,alpha_true)
    pred    = prediction(tt,res.x)
    #
    truth   = ["{:10.5f}".format(x) for x in truth]
    pred    = ["{:10.5f}".format(x) for x in pred]
    #
    outstr  = "x({:6.2f})=".format(tt)
    for itm in truth: outstr += itm
    outstr += " vs predicted "
    for itm in pred: outstr += itm
    print(outstr)

How many function evaluations did the minimizer take when it was able to make use of gradient information?

# The End